# Run Backtest - Crypto Spread Arbitrage

## Global Imports

In [1]:
import itertools

import logging
import os
import pickle
from datetime import date, datetime, timedelta

import cryptomart as cm
import numpy as np
import pandas as pd
import pyutil
import requests
import vectorbt as vbt

import app
from app import bq_util
from app.enums import Exchange, InstrumentType, Interval, OHLCVColumn, SpreadColumn, Symbol
from app.errors import BigQueryError, ExchangeAPIError, MissingDataError, NotSupportedError
from app.feeds import Spread
from app.globals import STUDY_END_DATE, STUDY_INST_TYPES, STUDY_START_DATE, STUDY_TIME_RANGE

# Global APIs / Constants
cm_client = cm.Client(quiet=True)

# We may have different identifiers for different strategies / different datasets used
backtest_identifier = "spread_arb_v1"
z_score_period = 30
data_start_filter = pd.to_datetime(datetime(2022, 4, 9))


## Prepare Data

### Instrument Level Data

In [2]:
@pyutil.cache.cached("/tmp/cache/all_order_book_stats", refresh=False, identifiers=[backtest_identifier])
def all_order_book_stats() -> pd.DataFrame:
    all_stats = pd.DataFrame(index=pd.MultiIndex.from_arrays([[], [], []], names=["exchange", "inst_type", "symbol"]))

    for exchange in Exchange:
        for inst_type in STUDY_INST_TYPES:
            instruments = cm_client.instrument_info(exchange, inst_type)
            for symbol in Symbol:
                if symbol not in instruments["cryptomart_symbol"].to_list():
                    continue
                try:
                    instrument_stats = bq_util.get_order_book_stats(
                        exchange, symbol, app.enums.InstrumentType.PERP, cache_kwargs={"disabled": False}
                    )
                    instrument_stats.rename(columns=lambda c: f"order_book_{c}", inplace=True)
                except BigQueryError:
                    # Instrument does not exist in BigQuery database
                    continue
                all_stats.at[(exchange, inst_type, symbol), instrument_stats.columns] = instrument_stats.iloc[0].values

    return all_stats


@pyutil.cache.cached("/tmp/cache/all_bid_ask_spreads", refresh=False, identifiers=[backtest_identifier])
def all_bid_ask_spreads() -> pd.DataFrame:
    ba_spreads = pd.DataFrame(index=pd.MultiIndex.from_arrays([[], [], []], names=["exchange", "inst_type", "symbol"]))

    for exchange in Exchange:
        for inst_type in STUDY_INST_TYPES:
            instruments = cm_client.instrument_info(exchange, inst_type)
            for symbol in Symbol:
                if symbol not in instruments["cryptomart_symbol"].to_list():
                    continue
                try:
                    ba_spread = bq_util.get_bid_ask_spread(
                        exchange,
                        symbol,
                        app.enums.InstrumentType.PERP,
                        cache_kwargs={"disabled": False, "refresh": True},
                    )
                except BigQueryError:
                    # Instrument does not exist in BigQuery database
                    continue
                ba_spreads.at[(exchange, inst_type, symbol), "bid_ask_spread"] = pickle.dumps(ba_spread)

    return ba_spreads


@pyutil.cache.cached("/tmp/cache/all_funding_rates", refresh=False, identifiers=[backtest_identifier])
def all_funding_rates() -> pd.DataFrame:
    funding_rates = pd.DataFrame(
        index=pd.MultiIndex.from_arrays([[], [], []], names=["exchange", "inst_type", "symbol"])
    )

    for exchange in Exchange:
        for inst_type in STUDY_INST_TYPES:
            instruments = cm_client.instrument_info(exchange, inst_type)
            for symbol in Symbol:
                if symbol not in instruments["cryptomart_symbol"].to_list():
                    continue
                try:
                    fr = cm_client.funding_rate(exchange, symbol, *STUDY_TIME_RANGE, cache_kwargs={"disabled": False})
                except NotSupportedError:
                    # Instrument not supported by exchange
                    continue
                funding_rates.at[(exchange, inst_type, symbol), "funding_rate"] = pickle.dumps(fr)

    return funding_rates


@pyutil.cache.cached("/tmp/cache/all_ohlcv", refresh=False, identifiers=[backtest_identifier])
def all_ohlcv() -> pd.DataFrame:
    ohlcvs = pd.DataFrame(index=pd.MultiIndex.from_arrays([[], [], []], names=["exchange", "inst_type", "symbol"]))

    for exchange in Exchange:
        for inst_type in STUDY_INST_TYPES:
            instruments = cm_client.instrument_info(exchange, inst_type)
            for symbol in Symbol:
                if symbol not in instruments["cryptomart_symbol"].to_list():
                    continue
                try:
                    fr = cm_client.ohlcv(
                        exchange,
                        symbol,
                        inst_type,
                        list(map(lambda d: (d.year, d.month, d.day), [date(2022, 6, 19) - timedelta(days=z_score_period)]))[0],
                        STUDY_END_DATE,
                        cache_kwargs={"disabled": False},
                    )
                except NotSupportedError:
                    # Instrument not supported by exchange
                    continue
                ohlcvs.at[(exchange, inst_type, symbol), "ohlcv"] = pickle.dumps(fr)

    return ohlcvs


logging.getLogger("pyutil.cache").setLevel("INFO")
order_book_stats = all_order_book_stats()
funding_rates = all_funding_rates()
ohlcvs = all_ohlcv()
logging.getLogger("pyutil.cache").setLevel("INFO")
bid_ask_spreads = all_bid_ask_spreads()
logging.getLogger("pyutil.cache").setLevel("INFO")

instrument_data = (
    funding_rates.merge(bid_ask_spreads, left_index=True, right_index=True)
    .merge(ohlcvs, left_index=True, right_index=True)
    .merge(order_book_stats, left_index=True, right_index=True)
)


2022-07-18 00:04:18.046 INFO     pyutil.cache                                                                      log:35   Using cached value in call to all_order_book_stats((id:spread_arb_v1)) | key=9bb39db2d9e6661b0d6ffe4cf690a03e (/tmp/cache/all_order_book_stats)
2022-07-18 00:04:18.050 INFO     pyutil.cache                                                                      log:35   Using cached value in call to all_funding_rates((id:spread_arb_v1)) | key=9bb39db2d9e6661b0d6ffe4cf690a03e (/tmp/cache/all_funding_rates)
2022-07-18 00:04:18.671 INFO     pyutil.cache                                                                      log:35   Using cached value in call to all_ohlcv((id:spread_arb_v1)) | key=9bb39db2d9e6661b0d6ffe4cf690a03e (/tmp/cache/all_ohlcv)
2022-07-18 00:04:18.706 INFO     pyutil.cache                                                                      log:35   Using cached value in call to all_bid_ask_spreads((id:spread_arb_v1)) | key=9bb39db2d9e6661b0d6ffe4c

### Filter instruments by date and number of gaps

In [3]:
# TODO: parallelize me
for idx, row in instrument_data.iterrows():
    bid_ask_spread = pickle.loads(row.bid_ask_spread)
    funding_rate = pickle.loads(row.funding_rate)
    ohlcv = pickle.loads(row.ohlcv)

    # Filter start times using global data_start_filter
    bid_ask_spread = bid_ask_spread[bid_ask_spread.date >= data_start_filter]
    funding_rate = funding_rate[funding_rate.timestamp >= data_start_filter]
    ohlcv = ohlcv[ohlcv.open_time >= data_start_filter - pd.Timedelta(z_score_period, "days")]

    instrument_data.at[idx, "bid_ask_spread"] = pickle.dumps(bid_ask_spread)

    instrument_data.at[idx, "funding_rate"] = pickle.dumps(funding_rate)
    instrument_data.at[idx, "funding_rate_gaps"] = funding_rate.gaps
    instrument_data.at[idx, "funding_rate_first_date"] = funding_rate.earliest_time

    instrument_data.at[idx, "ohlcv"] = pickle.dumps(ohlcv)
    instrument_data.at[idx, "ohlcv_gaps"] = ohlcv.gaps
    instrument_data.at[idx, "ohlcv_first_date"] = ohlcv.earliest_time


# Drop rows with too many funding rate gaps / ohlcv gaps
fr_gap_thres = 3
ohlcv_gap_thres = 3
mask = (instrument_data.funding_rate_gaps <= instrument_data.funding_rate_gaps.std() * fr_gap_thres) & (
    instrument_data.ohlcv_gaps <= instrument_data.ohlcv_gaps.std() * ohlcv_gap_thres
)

instrument_data_dropped_rows = instrument_data[~mask]
instrument_data = instrument_data[mask]


### Fee and Margin Data

In [4]:
# Exchange-wide taker fee: https://www.binance.com/en/fee/futureFee
logging.getLogger("pyutil.cache").setLevel("WARNING")

# BINANCE
fee_info_binance = cm_client.binance.instrument_info("perpetual")
fee_info_binance = fee_info_binance.set_index("cryptomart_symbol").rename_axis(index="symbol")
fee_info_binance = (
    fee_info_binance[["maintMarginPercent", "requiredMarginPercent"]]
    .rename(columns={"maintMarginPercent": "maint_margin", "requiredMarginPercent": "init_margin"})
    .astype(float)
    .apply(lambda x: x / 100)
)
fee_info_binance = fee_info_binance.assign(fee_pct=0.0004, fee_fixed=0)


# BITMEX
fee_info_bitmex = cm_client.bitmex.instrument_info("perpetual")
fee_info_bitmex = fee_info_bitmex.set_index("cryptomart_symbol").rename_axis(index="symbol")
fee_info_bitmex = fee_info_bitmex[["maintMargin", "initMargin", "takerFee"]].rename(
    columns={"maintMargin": "maint_margin", "initMargin": "init_margin", "takerFee": "fee_pct"}
)
fee_info_bitmex = fee_info_bitmex.assign(fee_fixed=0)


# BYBIT
@pyutil.cache.cached("/tmp/cache/fee_margin_data", refresh=False, path_seperators=["exchange"])
def bybit_get_single_margin(exchange_symbol, exchange="bybit"):
    url = os.path.join(cm_client.bybit.base_url, "public", "linear", "risk-limit")
    params = {"symbol": exchange_symbol}
    request = requests.Request("GET", url=url, params=params)
    response = cm_client.bybit.dispatcher.send_request(request)
    response = cm.interfaces.api.APIInterface.extract_response_data(
        response, ["result"], ["ret_code"], 0, ["ret_msg"], raw=True
    )
    risk_tiers = pd.DataFrame(response)
    lowest_risk_tier = risk_tiers[risk_tiers.limit == risk_tiers.limit.min()]
    return (
        lowest_risk_tier[["maintain_margin", "starting_margin"]]
        .rename(columns={"maintain_margin": "maint_margin", "starting_margin": "init_margin"})
        .iloc[0]
    )


fee_info_bybit = cm_client.bybit.instrument_info("perpetual")
fee_info_bybit = fee_info_bybit.set_index("cryptomart_symbol").rename_axis(index="symbol")

fee_info_bybit = pd.concat([fee_info_bybit, fee_info_bybit.exchange_symbol.apply(bybit_get_single_margin)], axis=1)
fee_info_bybit = fee_info_bybit[["maint_margin", "init_margin", "taker_fee"]].rename(columns={"taker_fee": "fee_pct"})
fee_info_bybit = fee_info_bybit.assign(fee_fixed=0)


# COINFLEX
# CoinFLEX does not provide information on maint margin / initial margin. Set to NaN and fill with the average of the other exchanges
# Exchange-wide taker fee: https://coinflex.com/fees/
fee_info_coinflex = cm_client.coinflex.instrument_info("perpetual")
fee_info_coinflex = fee_info_coinflex.set_index("cryptomart_symbol").rename_axis(index="symbol")[[]]
fee_info_coinflex = fee_info_coinflex.assign(maint_margin=np.nan, init_margin=np.nan)
fee_info_coinflex = fee_info_coinflex.assign(fee_pct=0.0005, fee_fixed=0)


# FTX
# FTX has a complex margin scheme. Set to NaN and fill with the average of the other exchanges
# Exchange-wide taker fee: https://help.ftx.com/hc/en-us/articles/360024479432-Fees
fee_info_ftx = cm_client.ftx.instrument_info("perpetual")
fee_info_ftx = fee_info_ftx.set_index("cryptomart_symbol").rename_axis(index="symbol")[[]]
fee_info_ftx = fee_info_ftx.assign(maint_margin=np.nan, init_margin=np.nan)
fee_info_ftx = fee_info_ftx.assign(fee_pct=0.0007, fee_fixed=0)


# GATEIO
# gateio does not provide init margin rate. Set to NaN and fill with the average of the other exchanges
fee_info_gateio = cm_client.gateio.instrument_info("perpetual")
fee_info_gateio = fee_info_gateio.set_index("cryptomart_symbol").rename_axis(index="symbol")
fee_info_gateio = fee_info_gateio[["taker_fee_rate", "maintenance_rate"]].rename(
    columns={"taker_fee_rate": "fee_pct", "maintenance_rate": "maint_margin"}
)
fee_info_gateio = fee_info_gateio.assign(init_margin=np.nan)
fee_info_gateio = fee_info_gateio.assign(fee_fixed=0)
fee_info_gateio = fee_info_gateio[["maint_margin", "init_margin", "fee_pct", "fee_fixed"]]


# KUCOIN
fee_info_kucoin = cm_client.kucoin.instrument_info("perpetual")
fee_info_kucoin = fee_info_kucoin.set_index("cryptomart_symbol").rename_axis(index="symbol")
fee_info_kucoin = fee_info_kucoin[["maintainMargin", "initialMargin", "takerFeeRate", "makerFixFee"]].rename(
    columns={
        "maintainMargin": "maint_margin",
        "initialMargin": "init_margin",
        "takerFeeRate": "fee_pct",
        "makerFixFee": "fee_fixed",
    }
)


# OKEX
# Exchange-wide taker fee: https://www.okx.com/fees
@pyutil.cache.cached("/tmp/cache/fee_margin_data", refresh=False, path_seperators=["exchange"])
def okex_get_single_margin(exchange_symbol, exchange="okex"):
    url = os.path.join(cm_client.okex.base_url, "api", "v5", "public", "position-tiers")
    params = {"instType": "SWAP", "tdMode": "isolated", "uly": exchange_symbol}
    request = requests.Request("GET", url=url, params=params)
    response = cm_client.okex.dispatcher.send_request(request)
    response = cm.interfaces.api.APIInterface.extract_response_data(
        response, ["data"], ["code"], "0", ["msg"], raw=True
    )
    risk_tiers = pd.DataFrame(response)
    return (
        risk_tiers.loc[risk_tiers.tier == "2", ["mmr", "imr"]]
        .rename(columns={"mmr": "maint_margin", "imr": "init_margin"})
        .iloc[0]
    )


fee_info_okex = cm_client.okex.instrument_info("perpetual")
fee_info_okex = fee_info_okex.set_index("cryptomart_symbol").rename_axis(index="symbol")
fee_info_okex = pd.concat([fee_info_okex, fee_info_okex.uly.apply(okex_get_single_margin)], axis=1)
fee_info_okex = fee_info_okex[["maint_margin", "init_margin"]]
fee_info_okex = fee_info_okex.assign(fee_pct=0.0005, fee_fixed=0)

logging.getLogger("pyutil.cache").setLevel("INFO")


# Join with instrument_data
DEFAULT_INIT_MARGIN = 0
DEFAULT_MAINT_MARGIN = 0.15
all_fee_info = pd.concat(
    [
        fee_info_binance,
        fee_info_bitmex,
        fee_info_bybit,
        fee_info_coinflex,
        fee_info_ftx,
        fee_info_gateio,
        fee_info_kucoin,
        fee_info_okex,
    ],
    keys=["binance", "bitmex", "bybit", "coinflex", "ftx", "gateio", "kucoin", "okex"],
    names=["exchange"],
)

all_fee_info = (
    all_fee_info.astype(float)
    .groupby("symbol")
    .apply(lambda c: c.fillna(c.mean()))
    .fillna({"init_margin": DEFAULT_INIT_MARGIN, "maint_margin": DEFAULT_MAINT_MARGIN})
)

instrument_data = instrument_data.join(all_fee_info).reorder_levels(instrument_data.index.names)


### Spread Level Data

In [16]:
instrument_data_crossed = (
    instrument_data.reset_index()
    .merge(instrument_data.reset_index(), how="cross", suffixes=("_a", "_b"))
    .pipe(
        lambda df: df[
            (df.exchange_a < df.exchange_b) & (df.inst_type_a <= df.inst_type_b) & (df.symbol_a == df.symbol_b)
        ]
    )
    .drop(columns="symbol_b")
    .rename(columns={"symbol_a": "symbol"})
    .reset_index(drop=True)
)

# TODO: parallelize me
spreads = pd.DataFrame(
    index=pd.MultiIndex.from_arrays(
        [[], [], [], [], []], names=["exchange_a", "exchange_b", "inst_type_a", "inst_type_b", "symbol"]
    )
)

for idx, row in instrument_data_crossed.iterrows():
    indexer_a = (row.exchange_a, row.inst_type_a, row.symbol)
    indexer_b = (row.exchange_b, row.inst_type_b, row.symbol)
    indexer = (row.exchange_a, row.exchange_b, row.inst_type_a, row.inst_type_b, row.symbol)
    spread = Spread.from_ohlcv(pickle.loads(row.ohlcv_a), pickle.loads(row.ohlcv_b))

    bid_ask_a = pickle.loads(bid_ask_spreads.at[indexer_a, "bid_ask_spread"])
    bid_ask_b = pickle.loads(bid_ask_spreads.at[indexer_b, "bid_ask_spread"])
    fr_a = pickle.loads(funding_rates.at[indexer_a, "funding_rate"])
    fr_b = pickle.loads(funding_rates.at[indexer_b, "funding_rate"])

    spread.add_bid_ask_spread(bid_ask_a, bid_ask_b)
    spread.add_funding_rate(fr_a, fr_b)

    spreads.at[indexer, "spread"] = pickle.dumps(spread)
    spreads.at[
        indexer, "alias"
    ] = f"{row.exchange_a[:4]}_{row.exchange_b[:4]}_{row.inst_type_a[:4]}_{row.inst_type_b[:4]}_{row.symbol}"
    spreads.at[indexer, "volatility"] = spread.volatility()
    spreads.at[indexer, (f"avg_ba_spread_{s}" for s in ("a", "b"))] = set(
        x.bid_ask_spread.mean() for x in (bid_ask_a, bid_ask_b)
    )
    spreads.at[indexer, "earliest_time"] = spread.earliest_time
    spreads.at[indexer, "latest_time"] = spread.latest_time
    spreads.at[indexer, "valid_rows"] = len(spread.valid_rows)
    spreads.at[indexer, "missing_rows"] = len(spread.missing_rows)
    spreads.at[indexer, "gaps"] = spread.gaps

    fee_info_keys = ["init_margin", "maint_margin", "fee_pct", "fee_fixed"]
    for key in ["a", "b"]:
        spreads.at[indexer, f"fee_info_{key}"] = pickle.dumps({k: getattr(row, f"{k}_{key}") for k in fee_info_keys})


# Optional: filter spreads by volatility, valid_rows, gaps, etc...


## Save Data for backtest

In [71]:
spreads.to_pickle("spreads.pkl")

## Run backtest

In [67]:
class BacktestResult:
    def __init__(self, portfolio: vbt.Portfolio, feed: Spread):
        self.portfolio = portfolio
        self.feed = feed

    def slippage(self):
        close_prices = self.feed.underlying_col("close").droplevel(1, axis=1).rename(columns=lambda c: c.split(".")[1])
        filled_prices = (
            self.portfolio.orders.records_readable.groupby(["Timestamp", "Column"])
            .first()
            .Price.unstack()
            .rename(columns=lambda c: c.split(".")[1])
        )
        sizes = (
            self.portfolio.orders.records_readable.groupby(["Timestamp", "Column"])
            .sum()
            .Size.unstack("Column")
            .rename(columns=lambda c: c.split(".")[1])
        )
        return abs(close_prices - filled_prices) * sizes

    def plot(self):
        portfolio = self.portfolio
        feed = self.feed

        df = portfolio.trades.records.sort_values(["entry_idx", "col"])
        # For column 0, a short means we are long on the spread and vice versa
        df = df[df.col == 0]

        # direction 1 = short
        # direction 0 = long
        long_trades = df[df.direction == 1]
        short_trades = df[df.direction == 0]

        temp_signals = np.zeros(len(feed))
        temp_signals[long_trades.entry_idx] = True
        long_entries = pd.Series(index=feed[feed.time_column], data=temp_signals).astype(bool)

        temp_signals = np.zeros(len(feed))
        temp_signals[long_trades.exit_idx] = True
        long_exits = pd.Series(index=feed[feed.time_column], data=temp_signals).astype(bool)

        temp_signals = np.zeros(len(feed))
        temp_signals[short_trades.entry_idx] = True
        short_entries = pd.Series(index=feed[feed.time_column], data=temp_signals).astype(bool)

        temp_signals = np.zeros(len(feed))
        temp_signals[short_trades.exit_idx] = True
        short_exits = pd.Series(index=feed[feed.time_column], data=temp_signals).astype(bool)

        fig = vbt.make_subplots(rows=8, cols=1, shared_xaxes=True, vertical_spacing=0.05)
        spread = feed.set_index(feed.time_column).close
        zscore = feed.zscore()

        spread.vbt.plot(add_trace_kwargs=dict(row=1, col=1), fig=fig, title=feed._underlying_info)
        zscore.vbt.plot(add_trace_kwargs=dict(row=2, col=1), fig=fig)

        # Plot entry and exit markers on z-score
        short_entries.vbt.signals.plot_as_exit_markers(
            zscore,
            add_trace_kwargs=dict(row=2, col=1),
            trace_kwargs=dict(marker=dict(opacity=0.4, size=12, color="green"), name="short_entry"),
            fig=fig,
        )
        short_exits.vbt.signals.plot_as_entry_markers(
            zscore,
            add_trace_kwargs=dict(row=2, col=1),
            trace_kwargs=dict(marker=dict(opacity=0.4, size=12, color="red"), name="short_exit"),
            fig=fig,
        )
        long_entries.vbt.signals.plot_as_entry_markers(
            zscore,
            add_trace_kwargs=dict(row=2, col=1),
            trace_kwargs=dict(marker=dict(opacity=0.8), name="long_entry"),
            fig=fig,
        )
        long_exits.vbt.signals.plot_as_exit_markers(
            zscore,
            add_trace_kwargs=dict(row=2, col=1),
            trace_kwargs=dict(marker=dict(opacity=0.8), name="long_exit"),
            fig=fig,
        )

        # Plot individual close prices
        feed.underlying_col("close").droplevel(1, axis=1).rename(
            columns=lambda c: c.split(".")[1] + " close price"
        ).vbt.plot(add_trace_kwargs=dict(row=3, col=1), fig=fig)

        # Plot daily returns
        (portfolio.returns() * 100).rename("% returns").vbt.scatterplot(add_trace_kwargs=dict(row=4, col=1), fig=fig)

        # Plot order entry and exit prices
        orders = portfolio.orders.records_readable
        orders["Side"] = orders["Side"].replace({"Sell": -1, "Buy": 1})
        orders["Price"] = orders["Price"] * orders["Side"]
        orders["When"] = pd.concat(
            [pd.Series(["entry", "exit"]).repeat(2)] * int(np.ceil(len(orders) / 4)), ignore_index=True
        ).values[: len(orders)]
        orders = orders.set_index(["Timestamp", "Column", "When"])
        orders = orders.unstack(["Column", "When"]).Price.rename(
            columns=lambda c: c.split(".")[1] + " fill price", level=0
        )
        orders.vbt.scatterplot(add_trace_kwargs=dict(row=5, col=1), fig=fig)

        # Plot daily PnL
        res.portfolio.trades.records_readable.sort_values("Entry Timestamp").groupby(
            "Entry Timestamp"
        ).PnL.sum().reindex(res.feed.open_time).fillna(0).rename("PnL").vbt.scatterplot(
            add_trace_kwargs=dict(row=6, col=1), fig=fig
        )

        # Plot cumulative returns
        (portfolio.cumulative_returns() * 100).rename("cumulative returns").vbt.plot(
            add_trace_kwargs=dict(row=7, col=1), fig=fig
        )

        # Plot slippage
        self.slippage().vbt.scatterplot(add_trace_kwargs=dict(row=8, col=1), fig=fig)

        fig.update_layout(height=1200, width=1800, hovermode="x unified", hoverlabel={"namelength": -1}, legend=None)
        fig.add_shape(
            type="rect",
            xref="paper",
            yref="y2",
            x0=0,
            y0=1,
            x1=1,
            y1=-1,
            fillcolor="gray",
            opacity=0.2,
            layer="below",
            line_width=0,
        )

        return fig

    def analyze(self):
        display(self.plot())
        display(f"slippage: {self.slippage().sum().sum()}")
        display(self.portfolio.stats())
        display(self.portfolio.trades.records_readable.sort_values("Entry Timestamp").head(10))
        display(self.portfolio.orders.records_readable.sort_values("Timestamp").head(20))
        display(self.feed.underlyings)


class BacktestRunner:
    def __init__(
        self,
        initial_cash=150000,
        trade_value=10000,
        vbt_function=app.vbt_backtest.from_order_func_wrapper,
        z_score_period=z_score_period,
        z_score_thresholds=(0, 1),  # (entry_threshold, exit_threshold)
        log_dir=None,
        force_logging=False,
    ):
        self.initial_cash = initial_cash
        self.trade_value = trade_value
        self.vbt_function = vbt_function
        self.z_score_period = z_score_period
        self.z_score_thresholds = z_score_thresholds

        self.logging = force_logging or (log_dir is not None)
        if log_dir is not None:
            self.log_dir = self.make_log_dir(log_dir)

    @staticmethod
    def make_log_dir(path):
        time_now = datetime.now()
        date_now = time_now.date().strftime("%Y-%m-%d")
        hour_now = time_now.strftime("%H")
        minute_now = time_now.strftime("%M")
        full_log_dir = os.path.join(path, date_now, hour_now, minute_now)
        os.makedirs(full_log_dir, exist_ok=True)
        return full_log_dir

    @staticmethod
    def unique_file_name(path):
        i = 2
        while os.path.exists(path):
            path = f"{path}_{i}"
            i += 1
        return f"{path}.log"

    def _run_single_spread(self, row: pd.Series):
        spread = pickle.loads(row.spread)
        alias = row.alias

        close_prices = np.array(spread.underlying_col("close"))
        funding_rate = np.array(spread.underlying_col("funding_rate"))
        bid_ask_spread = np.array(spread.underlying_col("bid_ask_spread"))
        zscore = np.array(spread.zscore(period=self.z_score_period))
        var = tuple(zip(spread.value_at_risk(percentile=5), spread.value_at_risk(percentile=95)))

        fee_info = {}
        fee_info_a = pickle.loads(row.fee_info_a)
        fee_info_b = pickle.loads(row.fee_info_a)
        for key in ["init_margin", "maint_margin", "fee_pct", "fee_fixed"]:
            fee_info[key] = (fee_info_a[key], fee_info_b[key])

        bt_args = app.vbt_backtest.BacktestArgs(
            initial_cash=self.initial_cash,
            trade_value=self.trade_value,
            z_score_thresholds=self.z_score_thresholds,
            var=var,
            init_margin=fee_info["init_margin"],
            maint_margin=fee_info["maint_margin"],
            fee_pct=fee_info["fee_pct"],
            fee_fixed=fee_info["fee_fixed"],
            zscore=zscore,
            funding_rate=funding_rate,
            bid_ask_spread=bid_ask_spread,
            logging=self.logging,
        )
        if hasattr(self, "log_dir"):
            log_file_name = os.path.join(self.log_dir, self.unique_file_name(alias))
            bt_func = pyutil.io.redirect_stdout(log_file_name)(self.vbt_function)
        else:
            bt_func = self.vbt_function
        res = bt_func(close_prices, bt_args)
        res = res.replace(
            wrapper=res.wrapper.replace(
                index=spread.open_time, columns=spread.underlying_col("close").columns.get_level_values(0)
            )
        )
        return BacktestResult(res, spread)

    def run(self, spreads: pd.DataFrame, exchange_subset=[], inst_type_subset=["perpetual"], symbol_subset=[]):
        index_filter = pd.MultiIndex.from_product(
            [exchange_subset, exchange_subset, inst_type_subset, inst_type_subset, symbol_subset]
        )
        if len(index_filter) > 0:
            spreads = spreads.filter(index_filter, axis=0)

        results = spreads.apply(self._run_single_spread, axis=1)
        return results


In [64]:
show_columns = ["volatility", "avg_ba_spread_a", "avg_ba_spread_b", "ba_spread_a_err_symbol", "ba_spread_b_err_symbol", "ba_spread_err", "ba_spread_metric"]
spreads["ba_spread_err"] = abs(spreads["avg_ba_spread_a"] - spreads["avg_ba_spread_b"]) / spreads["avg_ba_spread_b"]
spreads["ba_spread_a_err_symbol"] = spreads.groupby("symbol").apply(lambda g: (g.avg_ba_spread_a - g.avg_ba_spread_a.mean()) / g.avg_ba_spread_a.mean()).reset_index(level=0, drop=True)
spreads["ba_spread_b_err_symbol"] = spreads.groupby("symbol").apply(lambda g: (g.avg_ba_spread_b - g.avg_ba_spread_b.mean()) / g.avg_ba_spread_b.mean()).reset_index(level=0, drop=True)
spreads["ba_spread_metric"] = (spreads["ba_spread_err"] + 1 - spreads.loc[:, ("ba_spread_a_err_symbol", "ba_spread_b_err_symbol")].mean(axis=1)) / 2
display(spreads[show_columns].sort_values("volatility"))
display(spreads[show_columns].sort_values("ba_spread_metric", ascending=False))

volatility  \
exchange_a exchange_b inst_type_a inst_type_b symbol               
binance    okex       perpetual   perpetual   BTC       0.517591   
                                              ETH       0.518931   
           gateio     perpetual   perpetual   BTC       0.598759   
           bybit      perpetual   perpetual   BTC       0.623668   
gateio     okex       perpetual   perpetual   BTC       0.654557   
...                                                          ...   
bitmex     okex       perpetual   perpetual   FTM     237.385773   
           kucoin     perpetual   perpetual   FTM     237.415312   
           bybit      perpetual   perpetual   FTM     237.496516   
           ftx        perpetual   perpetual   FTM     237.818531   
           gateio     perpetual   perpetual   FTM     237.997303   

                                                      avg_ba_spread_a  \
exchange_a exchange_b inst_type_a inst_type_b symbol                    
binance    okex       perpetual   perpetual   BTC            4.599322   
                                              ETH            0.469027   
           gateio     perpetual   perpetual   BTC            4.599322   
           bybit      perpetual   perpetual   BTC            9.629185   
gateio     okex       perpetual   perpetual   BTC            5.590290   
...                                                               ...   
bitmex     okex       perpetual   perpetual   FTM            0.092118   
           kucoin     perpetual   perpetual   FTM            0.092118   
           bybit      perpetual   perpetual   FTM            0.092118   
           ftx        perpetual   perpetual   FTM            0.092118   
           gateio     perpetual   perpetual   FTM            0.092118   

                                                      avg_ba_spread_b  \
exchange_a exchange_b inst_type_a inst_type_b symbol                    
binance    okex       perpetual   perpetual   BTC            6.367177   
                                              ETH            0.475798   
           gateio     perpetual   perpetual   BTC            5.590290   
           bybit      perpetual   perpetual   BTC            4.599322   
gateio     okex       perpetual   perpetual   BTC            6.367177   
...                                                               ...   
bitmex     okex       perpetual   perpetual   FTM            0.002177   
           kucoin     perpetual   perpetual   FTM            0.002015   
           bybit      perpetual   perpetual   FTM            0.006581   
           ftx        perpetual   perpetual   FTM            0.001533   
           gateio     perpetual   perpetual   FTM            0.003112   

                                                      ba_spread_a_err_symbol  \
exchange_a exchange_b inst_type_a inst_type_b symbol                           
binance    okex       perpetual   perpetual   BTC                  -0.816622   
                                              ETH                  -0.446752   
           gateio     perpetual   perpetual   BTC                  -0.816622   
           bybit      perpetual   perpetual   BTC                  -0.616078   
gateio     okex       perpetual   perpetual   BTC                  -0.777111   
...                                                                      ...   
bitmex     okex       perpetual   perpetual   FTM                   2.181912   
           kucoin     perpetual   perpetual   FTM                   2.181912   
           bybit      perpetual   perpetual   FTM                   2.181912   
           ftx        perpetual   perpetual   FTM                   2.181912   
           gateio     perpetual   perpetual   FTM                   2.181912   

                                                      ba_spread_b_err_symbol  \
exchange_a exchange_b inst_type_a inst_type_b symbol                           
binance    okex       perpetual   perpetual   BTC                  -

volatility  \
exchange_a exchange_b inst_type_a inst_type_b symbol               
coinflex   gateio     perpetual   perpetual   DOGE      1.198809   
           okex       perpetual   perpetual   DOGE      1.328771   
           ftx        perpetual   perpetual   DOGE      1.397844   
           kucoin     perpetual   perpetual   DOGE      1.619428   
binance    coinflex   perpetual   perpetual   DOGE      1.205729   
...                                                          ...   
bybit      coinflex   perpetual   perpetual   CRV       2.313184   
                                              UNI       2.059601   
                                              YFI       3.014751   
binance    coinflex   perpetual   perpetual   DOT       1.225305   
bybit      coinflex   perpetual   perpetual   DOT       1.280413   

                                                      avg_ba_spread_a  \
exchange_a exchange_b inst_type_a inst_type_b symbol                    
coinflex   gateio     perpetual   perpetual   DOGE           0.020147   
           okex       perpetual   perpetual   DOGE           0.020147   
           ftx        perpetual   perpetual   DOGE           0.020147   
           kucoin     perpetual   perpetual   DOGE           0.020147   
binance    coinflex   perpetual   perpetual   DOGE           0.020147   
...                                                               ...   
bybit      coinflex   perpetual   perpetual   CRV            0.019761   
                                              UNI            0.030230   
                                              YFI          128.238628   
binance    coinflex   perpetual   perpetual   DOT            0.024029   
bybit      coinflex   perpetual   perpetual   DOT            0.097849   

                                                      avg_ba_spread_b  \
exchange_a exchange_b inst_type_a inst_type_b symbol                    
coinflex   gateio     perpetual   perpetual   DOGE           0.000121   
           okex       perpetual   perpetual   DOGE           0.000126   
           ftx        perpetual   perpetual   DOGE           0.000143   
           kucoin     perpetual   perpetual   DOGE           0.000233   
binance    coinflex   perpetual   perpetual   DOGE           0.000243   
...                                                               ...   
bybit      coinflex   perpetual   perpetual   CRV            0.175076   
                                              UNI            0.488220   
                                              YFI          457.180462   
binance    coinflex   perpetual   perpetual   DOT            0.662734   
bybit      coinflex   perpetual   perpetual   DOT            0.662734   

                                                      ba_spread_a_err_symbol  \
exchange_a exchange_b inst_type_a inst_type_b symbol                           
coinflex   gateio     perpetual   perpetual   DOGE                  2.762408   
           okex       perpetual   perpetual   DOGE                  2.762408   
           ftx        perpetual   perpetual   DOGE                  2.762408   
           kucoin     perpetual   perpetual   DOGE                  2.762408   
binance    coinflex   perpetual   perpetual   DOGE                  2.762408   
...                                                                      ...   
bybit      coinflex   perpetual   perpetual   CRV                  -0.595669   
                                              UNI                  -0.738057   
                                              YFI                  -0.299872   
binance    coinflex   perpetual   perpetual   DOT                  -0.853118   
bybit      coinflex   perpetual   perpetual   DOT                  -0.401891   

                                                      ba_spread_b_err_symbol  \
exchange_a exchange_b inst_type_a inst_type_b symbol                           
coinflex   gateio     perpetual   perpetual   DOGE                 -

In [68]:
btrunner = BacktestRunner(log_dir=os.path.join("/home/stefano/development/active_dev", "logs", backtest_identifier))
# btrunner = BacktestRunner(force_logging=True)
indexer = ("bitmex", "kucoin", "perpetual", "perpetual", "BTC")
res = btrunner._run_single_spread(spreads.loc[indexer])

In [69]:
res.analyze()

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter…

'slippage: 4977.892788854542'

Start                               2022-03-10 00:00:00
End                                 2022-06-18 00:00:00
Period                                101 days 00:00:00
Start Value                                    150000.0
End Value                                 203182.331159
Total Return [%]                              35.454887
Benchmark Return [%]                          -51.56236
Max Gross Exposure [%]                              0.0
Total Fees Paid                             2938.911779
Max Drawdown [%]                               2.454858
Max Drawdown Duration                   2 days 00:00:00
Total Trades                                         40
Total Closed Trades                                  40
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                       77.5
Best Trade [%]                                  20.3974
Worst Trade [%]                                -

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,ohlcv.bitmex.perpetual.BTC,1.180454,2022-04-08,43309.413318,38.343562,2022-04-09,42379.592148,30.520354,1028.746761,0.020122,Short,Closed,0
20,20,ohlcv.kucoin.perpetual.BTC,1.212610,2022-04-08,42271.254864,38.443906,2022-04-09,42726.599397,46.858022,466.853347,0.009108,Long,Closed,20
1,1,ohlcv.bitmex.perpetual.BTC,1.216352,2022-04-11,42036.939934,38.348794,2022-04-12,39575.476368,27.103289,2928.554604,0.057275,Short,Closed,1
21,21,ohlcv.kucoin.perpetual.BTC,1.296829,2022-04-11,39523.786233,38.441685,2022-04-12,40045.425877,48.949043,589.086529,0.011493,Long,Closed,21
2,2,ohlcv.bitmex.perpetual.BTC,1.246431,2022-04-14,41042.908145,38.367856,2022-04-15,39987.506355,29.381243,1247.736125,0.024390,Short,Closed,2
22,22,ohlcv.kucoin.perpetual.BTC,1.283252,2022-04-14,39940.305205,38.440109,2022-04-15,40517.902581,47.996011,654.766899,0.012775,Long,Closed,22
3,3,ohlcv.bitmex.perpetual.BTC,1.184626,2022-04-18,39721.304740,35.291165,2022-04-19,40758.667923,44.212830,1149.383302,0.024426,Long,Closed,3
23,23,ohlcv.kucoin.perpetual.BTC,1.151904,2022-04-18,40769.709093,35.222081,2022-04-19,41505.125258,27.857427,-910.208034,-0.019381,Short,Closed,23
4,4,ohlcv.bitmex.perpetual.BTC,1.191431,2022-04-25,39549.971142,35.340796,2022-04-26,40305.275286,40.015716,824.536253,0.017498,Long,Closed,4
24,24,ohlcv.kucoin.perpetual.BTC,1.162536,2022-04-25,40396.706232,35.221980,2022-04-26,38143.632015,29.257520,2554.801240,0.054401,Short,Closed,24


,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,ohlcv.bitmex.perpetual.BTC,2022-04-08,1.180454,43309.413318,38.343562,Sell
1,1,ohlcv.kucoin.perpetual.BTC,2022-04-08,1.212610,42271.254864,38.443906,Buy
2,2,ohlcv.kucoin.perpetual.BTC,2022-04-09,1.212610,42726.599397,46.858022,Sell
3,3,ohlcv.bitmex.perpetual.BTC,2022-04-09,1.180454,42379.592148,30.520354,Buy
4,4,ohlcv.bitmex.perpetual.BTC,2022-04-11,1.216352,42036.939934,38.348794,Sell
5,5,ohlcv.kucoin.perpetual.BTC,2022-04-11,1.296829,39523.786233,38.441685,Buy
6,6,ohlcv.kucoin.perpetual.BTC,2022-04-12,1.296829,40045.425877,48.949043,Sell
7,7,ohlcv.bitmex.perpetual.BTC,2022-04-12,1.216352,39575.476368,27.103289,Buy
9,9,ohlcv.kucoin.perpetual.BTC,2022-04-14,1.283252,39940.305205,38.440109,Buy
8,8,ohlcv.bitmex.perpetual.BTC,2022-04-14,1.246431,41042.908145,38.367856,Sell


ohlcv.bitmex.perpetual.BTC                             \
                                 open     high      low    close   
open_time                                                          
2022-03-10                    38718.5  42570.0  38640.5  41918.0   
2022-03-11                    41918.0  42030.5  38522.5  39405.0   
2022-03-12                    39405.0  40222.0  38200.5  38726.5   
2022-03-13                    38726.5  39547.5  38620.5  38738.5   
2022-03-14                    38738.5  39295.5  37569.0  37759.5   
...                               ...      ...      ...      ...   
2022-06-14                    26599.0  26863.0  21921.0  22478.0   
2022-06-15                    22478.0  23350.5  20766.5  22140.0   
2022-06-16                    22140.0  22799.5  20090.5  22551.0   
2022-06-17                    22551.0  22955.0  20223.5  20373.0   
2022-06-18                    20373.0  21343.0  20245.0  20444.0   

                                                      \
                  volume bid_ask_spread funding_rate   
open_time                                              
2022-03-10  1.276174e+09      60.417590     0.000100   
2022-03-11  1.194797e+09      60.417590    -0.000001   
2022-03-12  1.182933e+09      60.417590    -0.000064   
2022-03-13  7.333940e+08      60.417590    -0.000085   
2022-03-14  9.126950e+08      60.417590    -0.000046   
...                  ...            ...          ...   
2022-06-14  3.290992e+09      64.541323     0.000100   
2022-06-15  1.577578e+09      71.769887     0.000100   
2022-06-16  1.512860e+09      71.616359     0.000100   
2022-06-17  1.203457e+09      54.353998     0.000100   
2022-06-18  7.777030e+08      88.430298     0.000100   

           ohlcv.kucoin.perpetual.BTC                                          \
                                 open     high      low    close       volume   
open_time                                                                       
2022-03-10                    41947.0  42032.0  38519.0  39396.0   53441788.0   
2022-03-11                    39396.0  40214.0  38207.0  38714.0   47438494.0   
2022-03-12                    38715.0  39457.0  38632.0  38783.0   15841770.0   
2022-03-13                    38783.0  39300.0  37480.0  37751.0   24106807.0   
2022-03-14                    37751.0  40000.0  37501.0  39662.0   37807732.0   
...                               ...      ...      ...      ...          ...   
2022-06-14                    22464.0  23450.0  20803.0  22122.0  177751809.0   
2022-06-15                    22122.0  22810.0  20101.0  22577.0  177197316.0   
2022-06-16                    22576.0  22988.0  20216.0  20380.0  135597787.0   
2022-06-17                    20380.0  21349.0  20234.0  20456.0  106797062.0   
2022-06-18                    20457.0  20770.0  17600.0  18951.0  153479415.0   

                                        
           bid_ask_spread funding_rate  
open_time                               
2022-03-10      15.269678     0.000100  
2022-03-11      15.269678    -0.000001  
2022-03-12      15.269678    -0.000064  
2022-03-13      15.269678    -0.000085  
2022-03-14      15.269678    -0.000046  
...                   ...          ...  
2022-06-14      20.621219     0.000100  
2022-06-15      20.377111     0.000100  
2022-06-16      18.491866     0.000100  
2022-06-17      16.195242     0.000100  
2022-06-18      18.233392     0.000100  

[101 rows x 14 columns]

In [ ]:
# DEBUG
for idx, row in spreads.iterrows():
    try:
        res = btrunner._run_single_spread(row)
    except:
        print("error: ", idx)


In [15]:
res.portfolio.total_return()

-0.001847834269375453

In [ ]:
results = btrunner.run(spreads).to_frame("result")

In [ ]:
results["total_return"] = results.result.apply(lambda e: e.portfolio.total_return())
results["total_profit"] = results.result.apply(lambda e: e.portfolio.total_profit())


In [ ]:
results.sort_values("total_return").quantile(0.99)

In [ ]:
results.sort_values("total_return")

In [ ]:
spreads.sort_values("volatility").tail(10)

In [17]:
res.analyze()

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter…

Start                         2022-04-09 00:00:00
End                           2022-06-18 00:00:00
Period                           71 days 00:00:00
Start Value                              150000.0
End Value                            149722.82486
Total Return [%]                        -0.184783
Benchmark Return [%]                   -55.617258
Max Gross Exposure [%]                   0.010232
Total Fees Paid                        370.620142
Max Drawdown [%]                         0.184783
Max Drawdown Duration            39 days 00:00:00
Total Trades                                   12
Total Closed Trades                            12
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 50.0
Best Trade [%]                           9.616435
Worst Trade [%]                         -9.736488
Avg Winning Trade [%]                    3.542311
Avg Losing Trade [%]                    -3.659827


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,ohlcv.binance.perpetual.BTC,1.350812,2022-05-11,29074.7,15.709778,2022-05-12,29020.7,15.680601,41.553455,0.001058,Short,Closed,0
6,6,ohlcv.okex.perpetual.BTC,1.350598,2022-05-11,29079.3,15.709778,2022-05-12,29042.8,15.690060,-80.696667,-0.002055,Long,Closed,6
1,1,ohlcv.binance.perpetual.BTC,1.257601,2022-05-15,31324.4,15.757438,2022-05-16,29866.7,15.024156,1802.423311,0.045754,Short,Closed,1
7,7,ohlcv.okex.perpetual.BTC,1.257741,2022-05-15,31320.9,15.757438,2022-05-16,29878.9,15.031973,-1844.452629,-0.046821,Long,Closed,7
2,2,ohlcv.binance.perpetual.BTC,1.299265,2022-05-19,30319.9,15.757438,2022-05-21,29427.4,15.293600,1128.543278,0.028648,Short,Closed,2
8,8,ohlcv.okex.perpetual.BTC,1.299265,2022-05-19,30319.9,15.757438,2022-05-21,29443.1,15.301760,-1170.255049,-0.029707,Long,Closed,8
3,3,ohlcv.binance.perpetual.BTC,1.345292,2022-05-26,29194.0,15.709778,2022-05-27,28623.2,15.402621,736.780107,0.018760,Short,Closed,3
9,9,ohlcv.okex.perpetual.BTC,1.345117,2022-05-26,29197.8,15.709778,2022-05-27,28633.5,15.406159,-790.165248,-0.020119,Long,Closed,9
4,4,ohlcv.binance.perpetual.BTC,1.350672,2022-06-10,29077.7,15.709778,2022-06-11,28416.7,15.352661,-923.856879,-0.023523,Long,Closed,4
10,10,ohlcv.okex.perpetual.BTC,1.349851,2022-06-10,29095.4,15.709778,2022-06-11,28427.8,15.349314,870.101236,0.022154,Short,Closed,10


,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,ohlcv.binance.perpetual.BTC,2022-05-11,1.350812,29074.7,15.709778,Sell
1,1,ohlcv.okex.perpetual.BTC,2022-05-11,1.350598,29079.3,15.709778,Buy
2,2,ohlcv.okex.perpetual.BTC,2022-05-12,1.350598,29042.8,15.690060,Sell
3,3,ohlcv.binance.perpetual.BTC,2022-05-12,1.350812,29020.7,15.680601,Buy
4,4,ohlcv.binance.perpetual.BTC,2022-05-15,1.257601,31324.4,15.757438,Sell
5,5,ohlcv.okex.perpetual.BTC,2022-05-15,1.257741,31320.9,15.757438,Buy
6,6,ohlcv.okex.perpetual.BTC,2022-05-16,1.257741,29878.9,15.031973,Sell
7,7,ohlcv.binance.perpetual.BTC,2022-05-16,1.257601,29866.7,15.024156,Buy
8,8,ohlcv.binance.perpetual.BTC,2022-05-19,1.299265,30319.9,15.757438,Sell
9,9,ohlcv.okex.perpetual.BTC,2022-05-19,1.299265,30319.9,15.757438,Buy


ohlcv.binance.perpetual.BTC                             \
                                  open     high      low    close   
open_time                                                           
2022-04-09                     42229.1  42796.7  42100.3  42727.7   
2022-04-10                     42727.8  43399.4  41838.0  42140.4   
2022-04-11                     42140.3  42399.9  39136.7  39505.6   
2022-04-12                     39505.6  40700.0  39224.0  40060.7   
2022-04-13                     40060.7  41580.0  39569.8  41129.8   
...                                ...      ...      ...      ...   
2022-06-14                     22471.4  23424.2  20823.0  22122.7   
2022-06-15                     22122.7  22800.0  20100.2  22567.5   
2022-06-16                     22567.5  23000.0  20222.0  20387.4   
2022-06-17                     20387.4  21388.0  20228.0  20457.3   
2022-06-18                     20457.3  20768.4  17593.2  18959.7   

                                                    ohlcv.okex.perpetual.BTC  \
                 volume bid_ask_spread funding_rate                     open   
open_time                                                                      
2022-04-09   119722.654       4.753458     0.000100                  42250.9   
2022-04-10   181904.470       3.635435     0.000062                  42738.5   
2022-04-11   439530.020       4.506989     0.000100                  42147.8   
2022-04-12   349883.631       4.767611    -0.000070                  39530.8   
2022-04-13   324910.931       3.934481    -0.000015                  40070.3   
...                 ...            ...          ...                      ...   
2022-06-14  1247609.796       6.531293    -0.000052                  22479.4   
2022-06-15  1509301.169       4.134015    -0.000021                  22139.1   
2022-06-16   882634.220       5.105210    -0.000041                  22583.6   
2022-06-17   776153.257       3.300969    -0.000022                  20398.5   
2022-06-18  1268591.388       4.227956     0.000052                  20468.7   

                                                                              
               high      low    close     volume bid_ask_spread funding_rate  
open_time                                                                     
2022-04-09  42798.2  42122.0  42738.4   43852.19       4.548822     0.000100  
2022-04-10  43439.9  41848.4  42147.3   66697.91       4.079083     0.000062  
2022-04-11  42415.2  39145.0  39529.3  156532.92       4.493623     0.000100  
2022-04-12  40696.8  39251.2  40070.3  118326.41       6.100573    -0.000070  
2022-04-13  41570.7  39586.0  41149.3   85490.25       4.259350    -0.000015  
...             ...      ...      ...        ...            ...          ...  
2022-06-14  23374.3  20827.4  22139.1  358269.46       9.694330    -0.000052  
2022-06-15  22810.0  20080.0  22586.4  423504.85       7.994150    -0.000021  
2022-06-16  23000.0  20210.0  20397.2  206113.61       6.321477    -0.000041  
2022-06-17  21368.8  20233.2  20469.8  173559.94       7.328994    -0.000022  
2022-06-18  20778.0  17580.0  18972.5  286913.86       6.436993     0.000052  

[71 rows x 14 columns]

In [ ]:
pickle.loads(spreads.loc[indexer].spread).underlying_col("bid_ask_spread").reset_index().round(5)

In [ ]:
pickle.loads(spreads.loc[indexer].spread).plot()

In [ ]:
pickle.loads(spreads.loc[indexer].spread).returns().plot(backend="plotly")